In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv('./ukiyo-e_crawler/data/ukiyo-e_artworks_details.csv')
data.shape

(177985, 8)

In [3]:
print(data.columns)

Index(['Image URL', 'Artist', 'Title', 'Date', 'Details', 'Source',
       'Description', 'Similar Prints'],
      dtype='object')


In [8]:
# get the number of non-null values in description column
data['Description'].notnull().sum()

82648

In [5]:
# find entries with a 'clock' in the description
data['Description'].str.contains('').sum()
# 

82648

In [10]:
# get only entries that have a non-null description
data_desc = data[data['Description'].notnull()]
data_desc.head(2)

,Image URL,Artist,Title,Date,Details,Source,Description,Similar Prints
0,https://data.ukiyo-e.org/bm/scaled/AN00418884_...,Okumura Masanobu,NaN,1711,https://www.britishmuseum.org/collection/objec...,http://www.britishmuseum.org/research/search_t...,Print. Sixth of series of twelve (first two mi...,[]
2,https://data.ukiyo-e.org/loc/scaled/02127v.jpg,Okumura Masanobu,Courtesan likened to the Chinese sage Zhang Gu...,"[1715, printed later]",http://www.loc.gov/pictures/collection/jpd/ite...,http://www.loc.gov/pictures/search/?sp=2&co=jp...,Print shows a courtesan portraying the Chinese...,[]


In [17]:
len(data_desc[data_desc['Description'].str.contains('clock')][['Description']])
# at least 25 of these are likely to be the same image, because they're the same description!

57

In [18]:
#NLP libraries
import spacy, nltk, gensim, sklearn
import pyLDAvis.gensim_models

nlp = spacy.load('en_core_web_sm')

c:\Users\marga\Anaconda3\envs\ada\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [25]:
descs = data_desc[['Description']]

# remove newlines
for _, d in descs.iterrows():
    d.Description = " ".join(d.Description.split())

descs.head()

descs_list = descs['Description'].tolist()

In [47]:
import nltk
# nltk.download('averaged_perceptron_tagger')

test = descs_list[0]
# print("Test", test)
doctest = nlp(test)

# Sentence splitting
sentences = [sent for sent in doctest.sents]
for i in range(0, 3):
    print(f'Sentence {i}', sentences[i], '\n')

# Tokenization
tokens = nltk.word_tokenize(test)
tags = nltk.pos_tag(tokens)
nouns = [word for (word, pos) in tags if pos[0:2] == 'NN']
print(nouns)

Sentence 0 Print. 

Sentence 1 Sixth of series of twelve (first two missing). 

Sentence 2 Korean embassy procession: 

['Print', 'Sixth', 'series', 'twelve', 'procession', 'men', 'hats', 'beards', 'horses', 'bows', 'arrows', 'blowing', 'shells', 'trumpets', 'walking', 'Woodblock', 'paper', 'Inscribed']


In [48]:
# redoing this for all descs

data_desc['Nouns'] = data_desc['Description']

for _, d in data_desc.iterrows():
    desc = " ".join(d.Description.split())
    tokens = nltk.word_tokenize(desc)
    tags = nltk.pos_tag(tokens)
    nouns = [word for (word, pos) in tags if pos[0:2] == 'NN']
    d.Nouns = nouns

data_desc.head()

C:\Users\marga\AppData\Local\Temp\ipykernel_19072\2226010252.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_desc['Nouns'] = data_desc['Description']


,Image URL,Artist,Title,Date,Details,Source,Description,Similar Prints,Nouns
0,https://data.ukiyo-e.org/bm/scaled/AN00418884_...,Okumura Masanobu,NaN,1711,https://www.britishmuseum.org/collection/objec...,http://www.britishmuseum.org/research/search_t...,Print. Sixth of series of twelve (first two mi...,[],"[Print, Sixth, series, twelve, procession, men..."
2,https://data.ukiyo-e.org/loc/scaled/02127v.jpg,Okumura Masanobu,Courtesan likened to the Chinese sage Zhang Gu...,"[1715, printed later]",http://www.loc.gov/pictures/collection/jpd/ite...,http://www.loc.gov/pictures/search/?sp=2&co=jp...,Print shows a courtesan portraying the Chinese...,[],"[Print, sage, Zhang, Guo, Lao, Chōkarō, horse,..."
3,https://data.ukiyo-e.org/bm/scaled/AN00420719_...,Okumura Masanobu,NaN,1686-1764 (artist),https://www.britishmuseum.org/collection/objec...,http://www.britishmuseum.org/research/search_t...,Woodblock print. Kabuki. The actor Sanjo Kanta...,[],"[Woodblock, print, Kabuki, actor, Sanjo, Kanta..."
12,https://data.ukiyo-e.org/bm/scaled/AN00420734_...,Okumura Masanobu,NaN,1686-1764 (artist),https://www.britishmuseum.org/collection/objec...,http://www.britishmuseum.org/research/search_t...,Woodblock print. Kabuki. Kaomise (debut) of ac...,[],"[Woodblock, print, Kabuki, Kaomise, debut, act..."
16,https://data.ukiyo-e.org/honolulu/scaled/7514.jpg,Okumura Masanobu,Three Young Men: A man playing a hand drum,c. 1751,http://www.honoluluacademy.org/art/10475,http://art.honolulumuseum.org/emuseum/view/obj...,This center panel of a triptych depicts a man ...,[],"[panel, triptych, man, hand, drum, floor, left..."
